In [1]:
import numpy as np
import pandas as pd 
import keras
import re
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping,ModelCheckpoint
# fix random seed for reproducibility
np.random.seed(26)

Using TensorFlow backend.


In [2]:
work_train= {} 
work_test = {}
pre_process=["tfidf_tsvd_100.csv","w2v_100.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"nw_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"nw_working_test_"+f)

In [3]:
y=pd.read_csv("../bases/new_training_variants.csv").Class.values - 1

In [4]:
magic_feat=pd.read_csv("../l2_meta_features/svd25_molecular_functions.csv")
magic_feat=magic_feat.drop(["Class","Gene","ID","Variation"],axis=1)
magic_train=magic_feat[:len(y)]
magic_test=magic_feat[len(y):]
magic_train=magic_train.reset_index(drop=True)
magic_test=magic_test.reset_index(drop=True)

In [5]:
ID_test=work_test["w2v_100"]["ID"]
ID_train=work_train["w2v_100"]["ID"]

In [6]:
deep_train=work_train["tfidf_tsvd_100"].merge(work_train["w2v_100"])
deep_train=deep_train.drop("ID",axis=1)
deep_test=work_test["tfidf_tsvd_100"].merge(work_test["w2v_100"])
deep_test=deep_test.drop("ID",axis=1)

In [7]:
deep_train_1=pd.concat((deep_train,magic_train),axis=1)
deep_test_1=pd.concat((deep_test,magic_test),axis=1)

In [10]:
y_cat=to_categorical(y,9)

In [15]:
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=26)
cvscores = []
for train, test in kfold.split(np.array(deep_train_1), y):
  # create model
    model = Sequential()
    model.add(Dense(30, input_dim=263, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30,activation="relu"))
    model.add(Dense(9, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    es=EarlyStopping(monitor="loss",mode="min",patience=100)
    checkpoint=ModelCheckpoint(filepath="checknn",save_best_only=True,monitor="val_loss",mode="min")
    model.fit(np.array(deep_train_1)[train], y_cat[train],epochs=100,batch_size=50,validation_split=0.2,callbacks=[es,checkpoint],verbose=True)
    model.load_weights("checknn")
    # evaluate the model
    scores = model.evaluate(np.array(deep_train_1)[test], y_cat[test])
    print("%s: %.2f" % (model.metrics_names[0], scores[0]))
    cvscores.append(scores[0])
print("%.2f (+/- %.2f)" % (np.mean(cvscores), np.std(cvscores)))

Train on 2356 samples, validate on 590 samples
Epoch 1/100
2356/2356 [==============================] - 0s - loss: 12.9997 - acc: 0.1927 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 2/100
2356/2356 [==============================] - 0s - loss: 12.7489 - acc: 0.2076 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 3/100
2356/2356 [==============================] - 0s - loss: 12.5718 - acc: 0.2194 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 4/100
2356/2356 [==============================] - 0s - loss: 12.5183 - acc: 0.2228 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 5/100
2356/2356 [==============================] - 0s - loss: 12.5937 - acc: 0.2186 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 6/100
2356/2356 [==============================] - 0s - loss: 12.5469 - acc: 0.2216 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 7/100
2356/2356 [==============================] - 0s - loss: 12.5538 - acc: 0.2211 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 8/100
2356/2356 [==============================] - 

2356/2356 [==============================] - 0s - loss: 12.5401 - acc: 0.2220 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 65/100
2356/2356 [==============================] - 0s - loss: 12.5469 - acc: 0.2216 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 66/100
2356/2356 [==============================] - 0s - loss: 12.5401 - acc: 0.2220 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 67/100
2356/2356 [==============================] - 0s - loss: 12.5401 - acc: 0.2220 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 68/100
2356/2356 [==============================] - 0s - loss: 12.5401 - acc: 0.2220 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 69/100
2356/2356 [==============================] - 0s - loss: 12.5333 - acc: 0.2224 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 70/100
2356/2356 [==============================] - 0s - loss: 12.5401 - acc: 0.2220 - val_loss: 14.0145 - val_acc: 0.1305
Epoch 71/100
2356/2356 [==============================] - 0s - loss: 12.5401 - acc: 0.2220 - val_loss: 14.0145

2359/2359 [==============================] - 0s - loss: 11.9000 - acc: 0.2611 - val_loss: 10.0806 - val_acc: 0.3746
Epoch 27/100
2359/2359 [==============================] - 0s - loss: 11.8879 - acc: 0.2607 - val_loss: 10.0806 - val_acc: 0.3746
Epoch 28/100
2359/2359 [==============================] - 0s - loss: 11.8548 - acc: 0.2645 - val_loss: 10.0806 - val_acc: 0.3746
Epoch 29/100
2359/2359 [==============================] - 0s - loss: 11.8417 - acc: 0.2649 - val_loss: 10.0806 - val_acc: 0.3746
Epoch 30/100
2359/2359 [==============================] - 0s - loss: 11.8857 - acc: 0.2624 - val_loss: 10.0533 - val_acc: 0.3763
Epoch 31/100
2359/2359 [==============================] - 0s - loss: 11.8405 - acc: 0.2649 - val_loss: 10.0533 - val_acc: 0.3763
Epoch 32/100
2359/2359 [==============================] - 0s - loss: 11.8150 - acc: 0.2666 - val_loss: 10.0533 - val_acc: 0.3763
Epoch 33/100
2359/2359 [==============================] - 0s - loss: 11.8929 - acc: 0.2611 - val_loss: 10.0486

2359/2359 [==============================] - 0s - loss: 11.8098 - acc: 0.2662 - val_loss: 10.1105 - val_acc: 0.3712
Epoch 90/100
2359/2359 [==============================] - 0s - loss: 11.8943 - acc: 0.2599 - val_loss: 10.0283 - val_acc: 0.3763
Epoch 91/100
2359/2359 [==============================] - 0s - loss: 12.5204 - acc: 0.2196 - val_loss: 10.0533 - val_acc: 0.3763
Epoch 92/100
2359/2359 [==============================] - 0s - loss: 11.9672 - acc: 0.2543 - val_loss: 10.0113 - val_acc: 0.3763
Epoch 93/100
2359/2359 [==============================] - 0s - loss: 11.8985 - acc: 0.2603 - val_loss: 10.0171 - val_acc: 0.3763
Epoch 94/100
2359/2359 [==============================] - 0s - loss: 11.8166 - acc: 0.2658 - val_loss: 10.0260 - val_acc: 0.3780
Epoch 95/100
2359/2359 [==============================] - 0s - loss: 11.8232 - acc: 0.2662 - val_loss: 10.0260 - val_acc: 0.3780
Epoch 96/100
2359/2359 [==============================] - 0s - loss: 11.8077 - acc: 0.2671 - val_loss: 10.0117

2361/2361 [==============================] - 0s - loss: 11.8990 - acc: 0.2613 - val_loss: 10.0090 - val_acc: 0.3790
Epoch 52/100
2361/2361 [==============================] - 0s - loss: 11.8758 - acc: 0.2630 - val_loss: 10.0090 - val_acc: 0.3790
Epoch 53/100
2361/2361 [==============================] - 0s - loss: 11.8580 - acc: 0.2643 - val_loss: 10.0090 - val_acc: 0.3790
Epoch 54/100
2361/2361 [==============================] - 0s - loss: 11.8855 - acc: 0.2626 - val_loss: 10.0090 - val_acc: 0.3790
Epoch 55/100
2361/2361 [==============================] - 0s - loss: 11.8798 - acc: 0.2626 - val_loss: 10.0090 - val_acc: 0.3790
Epoch 56/100
2361/2361 [==============================] - 0s - loss: 11.8650 - acc: 0.2639 - val_loss: 10.0090 - val_acc: 0.3790
Epoch 57/100
2361/2361 [==============================] - 0s - loss: 11.8445 - acc: 0.2651 - val_loss: 10.0090 - val_acc: 0.3790
Epoch 58/100
2361/2361 [==============================] - 0s - loss: 11.8788 - acc: 0.2630 - val_loss: 10.0090

2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 14/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 15/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 16/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 17/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 18/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 19/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 20/100
2363/2363 [==============================] - 0s - loss: 12.5234 - acc: 0.2230 - val_loss: 14.1272

2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 77/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 78/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 79/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 80/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 81/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 82/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 83/100
2363/2363 [==============================] - 0s - loss: 12.5166 - acc: 0.2234 - val_loss: 14.1272

2364/2364 [==============================] - 0s - loss: 12.4982 - acc: 0.2242 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 39/100
2364/2364 [==============================] - 0s - loss: 12.5003 - acc: 0.2242 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 40/100
2364/2364 [==============================] - 0s - loss: 12.5139 - acc: 0.2229 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 41/100
2364/2364 [==============================] - 0s - loss: 12.5249 - acc: 0.2229 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 42/100
2364/2364 [==============================] - 0s - loss: 12.5069 - acc: 0.2238 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 43/100
2364/2364 [==============================] - 0s - loss: 12.5105 - acc: 0.2234 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 44/100
2364/2364 [==============================] - 0s - loss: 12.5296 - acc: 0.2225 - val_loss: 14.1272 - val_acc: 0.1235
Epoch 45/100
2364/2364 [==============================] - 0s - loss: 12.5387 - acc: 0.2217 - val_loss: 14.1272